## Is It a Car, Plane, or Ship?

Dataset Source: https://www.kaggle.com/datasets/abtabm/multiclassimagedatasetairplanecar

##### Import Necessary Libraries

In [0]:
import pyspark
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import DoubleType, ArrayType, StringType
import pyspark.sql.functions as F
from pyspark.ml import Pipeline

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

##### Versions of Libraries, Modules, Frameworks Used in This Project

In [0]:
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 4.2.6
Apache Spark version: 3.3.0


#### Create Functions Used Throughout Project

##### Function to Ingest Dataset

In [0]:
def ingest_folder_of_images_w_label(image_folders: [str], \
                                   labels: [str]):
    '''
    This function thats in a list of folder names that contain 
    images in it (not a list of folders with images in them) 
    and returns those images into a DataFrame.
    '''
    from pyspark.sql.dataframe import DataFrame
    
    for x in image_folders:
        temp_df: DataFrame = spark.read \
                .format("image") \
                .option("dropInvalid", value = True) \
                .load(x)
        
        temp_df = temp_df.withColumn("label", F.lit(labels[image_folders.index(x)]))
        
        if image_folders.index(x) > 0:
            df = df.union(temp_df)
        elif image_folders.index(x) == 0:
            df: DataFrame = temp_df
    return df

##### Create Pipeline Function (Creates Stages, Pipeline, Fit, & Transform)

In [0]:
def image_clf_model(dataset: pyspark.sql.dataframe.DataFrame, \
                   pretrained: str = "none" \
                   ) -> pyspark.sql.dataframe.DataFrame:
    '''
    This function creates the pipeline stages, the pipeline, 
    and it fits and transforms the dataset
    '''
    
    from pyspark.ml import Pipeline
    
    import sparknlp
    from sparknlp.base import ImageAssembler
    from sparknlp.annotator import ViTForImageClassification
    
    # Image Assembler
    img_assembler = ImageAssembler() \
        .setInputCol("image") \
        .setOutputCol("image_assembler")
    
    # Image Classifier
    if pretrained == "none":
        img_clf = ViTForImageClassification \
            .pretrained() \
            .setInputCols("image_assembler") \
            .setOutputCol("class")
    elif pretrained != "none":
        img_clf = ViTForImageClassification \
            .pretrained(pretrained, "en") \
            .setInputCols("image_assembler") \
            .setOutputCol("class")
    
    image_clf_pipe = Pipeline().setStages([
        img_assembler,
        img_clf
    ])
    
    predictions = image_clf_pipe.fit(dataset).transform(dataset)
    
    return predictions

##### Function for Evaluating Metrics

In [0]:
def mc_evaluate_with_spark_metrics(dataset: pyspark.sql.dataframe.DataFrame, \
                                metrics: [str], \
                                model_name: str, \
                                label_col: str = "label", \
                                predictionCol: str = "prediction" \
                               ) -> None:
    '''
    Calculate & display metrics for a multiclass classification analysis.
    '''
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    
    print("+---------------------------------------------+")
    print("|  " + model_name.center(41) + "  |")
    print("+---------------------------------------------+")
    print("|   %s  |  %s   |" % ("Metric".rjust(20), "Value".ljust(14)))
    print("+---------------------------------------------+")
    for x in metrics:
        evaluator = MulticlassClassificationEvaluator(labelCol=label_col, \
                                                      predictionCol=predictionCol, \
                                                      metricName=x) 
        score = evaluator.evaluate(dataset)
        print("|   %s  |  %s   |" % (x.rjust(20), str(round(score, 6)).ljust(14)))
        print("+---------------------------------------------+")

#### Actual Project

##### Ingest Training Dataset

In [0]:
training_data_files = ["/FileStore/tables/Dataset/train/cars", 
              "/FileStore/tables/Dataset/train/airplanes",
              "/FileStore/tables/Dataset/train/ship", 
              "/FileStore/tables/Dataset/test/cars", 
              "/FileStore/tables/Dataset/test/airplanes",
              "/FileStore/tables/Dataset/test/ships"]

label_values = ["cars", "airplanes", "ship", "cars", "airplanes", "ship"]

df: DataFrame = ingest_folder_of_images_w_label(training_data_files, label_values)

display(df)

image,label
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars429.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars167.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars602.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars603.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars398.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars364.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars990.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars415.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars212.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars
"List(dbfs:/FileStore/tables/Dataset/train/cars/cars428.jpg, 1200, 1200, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cars


Output can only be rendered in Databricks

##### Prepare & Train Model as Well as Generate Predictions

In [0]:
predictions = image_clf_model(df)

image_classifier_vit_base_patch16_224 download started this may take some time.
Approximate size to download 309.7 MB
[OK!]


##### Prepare Predictions For Metrics Evaluation Function (Part 1)

In [0]:
preds = predictions.select(F.col("label").alias("label"), F.col("class.result").getItem(0).alias("prediction"))

preds = preds.persist()

label_converter = {"ship" : "0", "cars" : "1", "airplanes" : "2"}

preds = preds.replace(label_converter, subset=["label"])

##### Prepare Predictions For Metrics Evaluation Function (Part 2)

In [0]:
cars_converter = ["sports car", "grille", 
                  "Model T", "wagon", 
                  "limousine", "pickup", 
                  "truck", "jeep", 
                  "pickup", "vehicle", 
                  "car wheel", "taxi", 
                  "convertible", "minivan", 
                  "radiator", "police van", 
                  "cab"]

planes_converter = ["airplane", "airship", 
                   "airliner", "warplane", 
                   "plane", "wing"] 

ships_converter = ["ship", "schooner", 
                    "liner", "submarine", 
                    "aircraft carrier", "vessel", 
                    "trimaran", "dock", 
                    "lakeside", "container ship"]

preds = preds.withColumn('prediction', F.split(preds.prediction, ',').getItem(0))

preds = preds.withColumn("prediction", \
                         F.when(F.col("prediction").isin(cars_converter), F.lit("1"))\
                         .when(F.col("prediction").isin(planes_converter), F.lit("2")) \
                         .when(F.col("prediction").isin(ships_converter), F.lit("0")) \
                         .otherwise(F.lit("-1")))


#Converter String data type to Double data type
preds = preds.withColumn("label", F.col("label").cast(DoubleType())) \
            .withColumn("prediction", F.col("prediction").cast(DoubleType()))

display(preds)

label,prediction
1.0,1.0
1.0,-1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,-1.0


- Because some of the predictions have multiple guesses, it required a slightly different approach for converting strings to doubles for metrics evaluation. 
- I tried to give some leeway with results. For example, I counted car parts as cars, airplane parts as airplanes.

##### Calculate & Display Metrics

In [0]:
metrics_to_eval = ["accuracy", "f1", "weightedPrecision", "weightedRecall"]

mc_evaluate_with_spark_metrics(preds, \
                                metrics_to_eval, \
                                "Is it a Plane, Boat, or Car?", \
                                label_col = "label", \
                                predictionCol = "prediction")

+---------------------------------------------+
|         Is it a Plane, Boat, or Car?        |
+---------------------------------------------+
|                 Metric  |  Value            |
+---------------------------------------------+
|               accuracy  |  0.89             |
+---------------------------------------------+
|                     f1  |  0.935331         |
+---------------------------------------------+
|      weightedPrecision  |  0.994898         |
+---------------------------------------------+
|         weightedRecall  |  0.89             |
+---------------------------------------------+


##### End Spark Session

In [0]:
df = df.unpersist()
preds = preds.unpersist()

spark.stop()

### Notes & Other Takeaways From This Project
****
- This model (the base ViT model) outperformed (by a wide margin) a model trained for specifically these types of images.
****
- This was one of my first computer vision-related projects. It is a good start, but it is also just a start!
****